<a href="https://colab.research.google.com/github/ArfaShoaib/PIAIC-PROJECTS/blob/main/RAG_Project2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.2 MB/s eta 0:00:00


In [3]:

from google.colab import userdata
userdata.get('PINECONE_API_KEY')
from pinecone import Pinecone, ServerlessSpec



pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

In [4]:
import time

index_name = "rag-project"  # change if desired


pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
 )


index = pc.Index(index_name)

In [18]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from google.colab import userdata



os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [19]:
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [20]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [21]:
#Data Save
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]


In [22]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['768dc884-029c-4b2a-91ca-5b6b6d95b31e',
 '72a96adf-ffb6-4533-b5c2-cd4e25dea11b',
 'de229558-36d8-48f7-ac89-9e932dee75a3',
 '9621beb5-a0c1-44e1-8c21-aa9665289556',
 '8ffff74d-2c52-4ff3-86bf-524eafbee90b',
 '939fe6d3-ab3e-4767-ac8b-9c57543d8dee',
 '56a5e346-1b4b-44b6-ae6a-f859e18ca26a',
 'e565021f-a4fa-41d6-b80e-f9f171cd9caf',
 'eaef07b2-6956-4b7a-b62f-b96f7c898713',
 'e550d40b-e2e1-4156-873a-e2fa1d333d78']

In [26]:
#Data Reterive
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [27]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.667716] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [29]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [31]:
def answers(query:str):
  #Vector Search
  vector_results = vector_store.similarity_search(query , k=2)
  print(len(vector_results))

  #PassnTo Model Vector Result + User Query
  final_answer =llm.invoke(f"answer this user query: {query}, here are some refrences {vector_results}")


  return final_answer

In [32]:
answers("LangChain provides abstractions to make working with LLMs easy")

2


AIMessage(content="The provided text mentions LangChain as a framework that simplifies working with Large Language Models (LLMs).  However, the included references are tweets that don't directly support this claim. One tweet promotes a competing framework, LangGraph, and the other simply mentions using LangChain in a project.  Therefore, while the initial statement about LangChain's ease of use is plausible (it's a common claim about the framework), the provided references don't offer evidence to support it.\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-cd740cc2-b6b3-4253-a855-7fc8a33bba31-0', usage_metadata={'input_tokens': 151, 'output_tokens': 100, 'total_tokens': 251, 'input_token_details': {'cache_read': 0}})